# Capstone Project

## Problem Statement

The world of craft beer has been growing steadily since the late 1970s. Nowadays you can easily find hundreds of varieties by hundreds if not thousands of different brewers. Even in most supermarkets you can find dozens of different beers available where before it was not uncommon to only see a few.

Along with a greater interest in craft beer has come a greater interest in and particpation in homebrewing. There are huge organizations of people who are creating and innovating with their own beers. Some homebrewers work in the industry, while others are just enthusiasts. All share a love of trying

With this incredible growth in the craft beer marketplace, are commercial breweries making the same types of beers that homebrewers are? If not, there is a potential to increase sales by putting out products for which demand is high, but supply does not yet exist.

In this project I would like to:

1. Analyze what styles of homebrew recipes are being made using clustering based on data scraped from www.brewtoad.com

2. Compare generated cluseter parameters with BJCP guidelines to determine if any BJCP categories should be expanded or combined to more accurately reflect current homebrewing trends, or if the BJCP is missing any categories within the homebrewing community.

3. Write recommendations for the BJCP in developing next set of style guidelines.

---
## Gathering Data

Data is coming from 2 separate sources:
1. Brewtoad Scraper: This is a scraper I wrote to pull recipes from the webiste www.brewtoad.com.  This scraper is running on an AWS EC2 instance to gather data currently. It will get the name, style, base statistics, and full recipe details of each recipe it scans.
2. 2015 BJCP Guidelines: [These guidelines](https://www.bjcp.org/docs/2015_Guidelines_Beer.pdf) are the most recent officially published beer guidelines. This pdf document can be found in .csv form [here](https://www.bjcp.org/docs/2015_styleguide.xml)